In [2]:
# from bert_model_implementation_torch.model import BertModel
from bert_model_implementation_torch.tokenization import  _is_control,_is_whitespace,_is_punctuation
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
from keras.src.utils import to_categorical
from torch.utils.data import DataLoader
import shutil

d:\conda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
05/14/2024 20:16:04 - INFO - numexpr.utils -   Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
05/14/2024 20:16:04 - INFO - numexpr.utils -   NumExpr defaulting to 8 threads.


In [11]:
import conllu

# Define a sentence
sentence = "Apple is looking at buying U.K. startup for $1 billion"

# Tokenize the sentence
tokens = sentence.split()

# Create a list of CoNLL Token objects
conll_tokens = []
for i, token in enumerate(tokens, start=1):
    conll_tokens.append(conllu.Token(
        id=i, form=token, lemma="_", upos="_", xpos="_", feats="_", head="_", deprel="_", deps="_", misc="_"
    ))

# Create a CoNLL object
conll = conllu.TokenList(conll_tokens)

# Print the CoNLL format
print(conll.serialize())


1	Apple	_	_	_	_	_	_	_	_
2	is	_	_	_	_	_	_	_	_
3	looking	_	_	_	_	_	_	_	_
4	at	_	_	_	_	_	_	_	_
5	buying	_	_	_	_	_	_	_	_
6	U.K.	_	_	_	_	_	_	_	_
7	startup	_	_	_	_	_	_	_	_
8	for	_	_	_	_	_	_	_	_
9	$1	_	_	_	_	_	_	_	_
10	billion	_	_	_	_	_	_	_	_




In [17]:
from conllu import parse

# Sample sentence
sentence = "The quick brown fox jumps over the lazy dog."

# (Optional) Convert sentence to a spaCy doc (assuming you have spaCy installed)
# doc = nlp(sentence)

# Convert sentence (or spaCy doc) to CoNLL-U format (replace with your conversion logic)
conll_data = sentence  # Replace with your conversion logic (e.g., using spaCy)

# Parse the CoNLL-U data string
parsed_data = parse(conll_data)

# Access the tokens and their information in the parsed data object
for token in parsed_data:
  print(f"ID: {token.id}, FORM: {token.form}, UPOS: {token.upos}")


The quick brown fox jumps over the lazy dog.


In [3]:
data = pd.read_csv("../data/Book1.csv")

In [4]:
data.head()

,prompt,intent,classes
0,Can you update the sales figures in cells <Cel...,Update cell range,entry and manipulation
1,Please modify the data in range <Range>,Update cell range,entry and manipulation
2,I need to change the prices in cells <Range>,Update cell range,entry and manipulation
3,Could you update the inventory levels from <Ce...,Update cell range,entry and manipulation
4,Please adjust the budget numbers in range <Range>,Update cell range,entry and manipulation


In [5]:
config = {
    'max_len': 256,
    'batch_size': 8,
    'epochs': 10,
    'lr':1e-05,
    'out_first_layer': 768,
    'dropout_rate': 0.1,
    'model_dir':'bert-base-uncased',
    'ckpt_path': './ckpts',
    'ckpt_model_path': './experiments'
}

In [6]:
intent_list = data.intent.unique().tolist()

In [7]:
class intent_dataset:
    def __init__(self, df: pd.DataFrame, tokenizer: BertTokenizer, max_len: int):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.classes = data.intent.unique().tolist()
        self.y = df['intent']
        self.x = df['prompt']
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        title = str(self.x[index])
        title= ''.join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens = True,
            return_attention_mask = True,
            return_tensors = 'pt',
            return_token_type_ids = True,
            padding = 'max_length',
            max_length = self.max_len,
            truncation = True
        )
        # print(self.classes.index(self.y[index]))
        target = self.classes.index(self.y[index])  # Get the class index
        target_tensor = torch.zeros(len(self.classes),dtype= torch.float32)  # Initialize target tensor with zeros
        target_tensor[target] = 1  # Set the corresponding index to 1
        return {
            'input_ids': inputs["input_ids"].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'targets': target_tensor
        }

In [8]:
train = data.sample(frac=0.9, random_state=200).reset_index(drop=True)
val = data.drop(train.index).reset_index(drop=True)

In [9]:
tokenizer = BertTokenizer.from_pretrained(config["model_dir"])
train_dataset = intent_dataset(train, tokenizer, config["max_len"])
val_dataset = intent_dataset(val, tokenizer, config["max_len"])

In [10]:
train_loader = DataLoader(
    train_dataset,
    shuffle = True,
    batch_size = config["batch_size"],
    num_workers = 0
    )

val_loader = DataLoader(
    val_dataset,
    shuffle = False,
    batch_size = config["batch_size"],
    num_workers = 0
    )

In [11]:
train_loader.dataset[10]['targets']


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.])

In [12]:
len(train_loader.dataset[0]['input_ids'])


256

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [14]:
def load_ckpt(ckpt_path, model, optimizer):
    ckpt = torch.load(ckpt_path)
    model.load_state_dict(ckpt['state_dict'])
    optimizer.load_state_dict(ckpt['state_dict'])
    valid_loss_min = ckpt['valid_loss_min']
    return model, optimizer, ckpt['epoch'], valid_loss_min.item()

In [15]:
def save_ckpt(state, is_best, ckpt_path, best_model_path):
    f_path= ckpt_path
    torch.save(state, f_path)
    if is_best:
        best_f_path = best_model_path
        shutil.copyfile(f_path, best_f_path)

In [16]:
class intent_model(nn.Module):
    def __init__(self, config: dict, intent_labels: list[str], dropout: float = 0.1):
        super(intent_model,self).__init__()
        self.config = config
        self.intent_labels = intent_labels
        self.dropout_rate = dropout
        self.bert = BertModel.from_pretrained(self.config["model_dir"])
        self.dropout = nn.Dropout(self.dropout_rate)
        self.layer_1 = nn.Linear(self.bert.config.hidden_size,self.config["out_first_layer"])
        self.activation_1 = nn.ReLU()
        self.layer_2 = nn.Linear(self.config["out_first_layer"], len(self.intent_labels))
        
    def forward(self, input_ids: torch.Tensor, token_type_ids: torch.Tensor, attention_mask: torch.Tensor):
        output = self.bert(input_ids, token_type_ids, attention_mask)
        output_dropout = self.dropout(output.pooler_output)
        out_layer_1 = self.layer_1(output_dropout)
        act_1 = self.activation_1(out_layer_1)
        out_layer_2 = self.layer_2(act_1)
        return out_layer_2
        

In [17]:
model = intent_model(config, intent_list, 0.1)
model.to(device)

intent_model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [18]:
def loss_fn(outputs, targets):
    targets = targets.float()
    return nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.Adam(params = model.parameters(), lr = config["lr"])

In [19]:
def train(model, epochs, train_loader, val_loader, optimizer, ckpt_path, best_model_path):
    valid_loss_min = np.Inf
    for epoch in range(1, epochs + 1):
        train_loss = 0
        valid_loss = 0
        model.train()
        for batch_index, batch in tqdm(enumerate(train_loader)):
            input_ids = batch['input_ids'].to(device, dtype= torch.long)
            attention_mask = batch["attention_mask"].to(device, dtype= torch.long)
            token_type_ids = batch["token_type_ids"].to(device, dtype= torch.long)
            targets = batch["targets"].to(device, dtype= torch.long)
            outputs = model(input_ids, token_type_ids, attention_mask)
            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += (1/(batch_index + 1)) * (loss.item() - train_loss)
        print(f"epoch {epoch} ended with train loss of {train_loss}")
        model.eval()
        with torch.no_grad():
            for batch_index, batch in tqdm(enumerate(val_loader)):
                input_ids = batch['input_ids'].to(device, dtype= torch.long)
                attention_mask = batch["attention_mask"].to(device, dtype= torch.long)
                token_type_ids = batch["token_type_ids"].to(device, dtype= torch.long)
                targets = batch["targets"].to(device, dtype= torch.long)
                outputs = model(input_ids, token_type_ids, attention_mask)
                loss = loss_fn(outputs, targets)
                val_loss += (1/(batch_index + 1)) * (loss.item() - val_loss)
        print(f"epoch {epoch} ended with train loss of {valid_loss}")
        checkpoint = {
            'epoch': epoch +1,
            'valid_loss_min': val_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        save_ckpt(checkpoint, False, ckpt_path, best_model_path)
        return model

In [20]:
train = train(model, config["epochs"], train_loader, val_loader, optimizer, config["ckpt_path"], config["ckpt_model_path"])

4648it [2:00:21,  1.55s/it]


KeyboardInterrupt: 